# Extract Health Vials from Screenshots
Using a sample image ```../images/health-vial-mask.1440.png``` this notebook will apply the mask to screenshots and extract the health vial image.  
<br>
Additionally, the notebook will resize the image to a size just larger than the extracted health vial, and use a transparent background.

In [ ]:
%%capture
%pip install Pillow

In [ ]:
from PIL import Image

def extract_masked_content(input_image_path, output_image_path, mask_image_path='../images/health-vial-mask.1440.png'):
    # Open the input image
    img = Image.open(input_image_path).convert("RGBA")
    
    # Open the mask image and convert to greyscale
    mask = Image.open(mask_image_path).convert("L")  # Convert the mask to greyscale

    # Create a new image by applying the mask
    masked_data = []
    data = img.getdata()

    for index, item in enumerate(data):
        # Get mask pixel value (0: black, 255: white)
        mask_value = mask.getpixel((index % img.width, index // img.width))
        
        # If the mask pixel is white (255), keep the corresponding input image pixel
        if mask_value < 255:  # If not white, keep the pixel
            masked_data.append(item)  # Keep the original pixel
        else:
            masked_data.append((0, 0, 0, 0))  # Fully transparent pixel

    # Create a new image with the masked content
    masked_image = Image.new("RGBA", img.size)
    masked_image.putdata(masked_data)

    # Find the bounding box of the non-transparent pixels
    bbox = masked_image.getbbox()

    # Crop the image to the bounding box
    if bbox:
        cropped_image = masked_image.crop(bbox)

        # Create a new transparent image for resizing
        max_size = max(cropped_image.size)
        new_image = Image.new("RGBA", (max_size, max_size), (0, 0, 0, 0))  # Transparent background

        # Paste cropped image onto the center of the new transparent image
        new_image.paste(cropped_image, ((max_size - cropped_image.width) // 2, (max_size - cropped_image.height) // 2))

        # Save the result
        new_image.save(output_image_path, format="PNG")
    else:
        print("No visible content to extract.")

In [ ]:
import os

# Define the base directories
input_base_dir = os.path.expanduser('~/Desktop/results')
output_base_dir = os.path.expanduser('~/Desktop/health-vials')

# Create the output directory if it doesn't exist
os.makedirs(output_base_dir, exist_ok=True)

# Get a list of all files in the input directory
all_files = os.listdir(input_base_dir)

# Filter to get only PNG files
png_image_paths = [os.path.join(input_base_dir, f) for f in all_files if f.endswith('.png')]

# Process each PNG image
for input_image_path in png_image_paths:
    # Construct the output image path
    output_image_path = os.path.join(output_base_dir, f'extracted_{os.path.basename(input_image_path)}')
    
    extract_masked_content(input_image_path, output_image_path)
    print(f'Processing {input_image_path} => {output_image_path}')